# Programmierübung 1 zu *Grundlagen der Optimierung* (WS2023)

## Einführung

### Verantwortlich
* Dr. Evelyn Herberg
* M.Sc. Masoumeh Hashemi
* B.Sc. Viktor Stein

### Zielsetzung
Das Ziel dieses *Jupyter Notebooks* ist es, Ihnen das Verhalten der im Kapitel 1 des Skripts vorgestellten Algorithmen zur Lösung allgemeiner, unrestringierter Optimierungsprobleme nahezubringen.
Wir werden den Einfluss der *Vorkonditionierer* im *Gradientenverfahren* untersuchen.

### Zur Nutzung des Notebooks
Die numerische Umsetzung der Verfahren und die graphische Visualisierung typischer Ergebnisse ist ein essentieller Baustein auf dem Weg zu einem ausgereiften Verständnis der Algorithmen.
Um programmiertechnische Schwierigkeiten weitestgehend auszuschließen, haben wir einiges an Code für Sie vorbereitet.
An den Schlüsselstellen der jeweiligen Implementierungen wurde der lauffähige Code durch auskommentierte Blöcke der Art
```python
### TODO BEGIN ###
# Compute the preconditioned gradient and the square of its (preconditioner-induced) norm 
# gradient = ...
# norm2_gradient = ...
### TODO END ###
```
ersetzt, in denen Sie zwischen `### TODO BEGIN ###` und `### TODO END ###` die entsprechenden Anweisungen Variablen (in diesem Beispiel die Berechnung von `gradient` und `norm2_gradient`) entsprechend des Kommentars ausführen, um Lauffähigkeit wieder herzustellen.
Welche Berechnungen und Auswertungen an den jeweiligen Stellen benötigt werden, können Sie im Skript nachlesen.
Sie können natürlich, bevor Sie genau diese vorbenannten Variablen schreiben, auch eigene Variablen beschreiben.

Wenn sie den Code vervollständigt haben, werden Sie an geeigneter Stelle um die Interpretation der Ergebnisse gebeten.
In den entsprechenden Zellen ersetzen Sie "**TODO Ihre Antwort hier**" mit ihrer Antwort.

## Das Gradientenverfahren für quadratische Zielfunktionen
In diesem Abschnitt wollen wir das Verhalten des *Gradientenverfahrens* bei *quadratischen Zielfunktionen* mit symmetrischem, positiv definitem quadratischen Teil bei Verwendung der *exakten Schrittweitenbestimmung* untersuchen.
Wir wollen hier insbesondere den Einfluss des Vorkonditionierers auf die Konvergenzgeschwindigkeit untersuchen, denn
wie wir wissen kann man in diesem Fall die *q-lineare Konvergenz* des Gradientenverfahrens gegen den eindeutigen Minimierer des Problems beweisen und kann sogar den Vorfaktor explizit in Abhängigkeit von der *verallgemeinerten Konditionszahl* angeben.

### Implementierung des Gradientenverfahrens
In der nachfolgenden Zelle finden Sie die Funktion zum Gradientenverfahren mit den fehlenden Berechnungen.

**Aufgabe:** Vervollständigen Sie den Code und führen Sie die Zelle aus.

Beachten Sie, dass sowohl die verwendete Schrittweitenberechnung als auch die Wahl des Vorkonditionierers - mit gutem Grund - durch den/die AnwenderIn explizit vorgegeben werden muss.

In [ ]:
# This module implements the preconditioned gradient scheme.

import numpy as np

def gradient_descent(f, x0, step_length_rule, preconditioner, parameters = {}):
  """ 
  Solve an unconstrained minimization problem using the preconditioned
  gradient descent method.

  Accepts:  
                   f: the objective function to be minimized
                  x0: the initial guess (list or numpy array with ndim == 1)
    step_length_rule: a step length computation function 
      preconditioner: a symmetric positive definite matrix (numpy array with ndim == 2)
          parameters: optional parameters (dictionary);
                      the following key/value pairs are evaluated:
                                ["atol_x"]: absolute stopping tolerance for the norm of updates in x
                                ["rtol_x"]: relative stopping tolerance for the norm of updates in x
                                ["atol_f"]: absolute stopping tolerance for the progress in the values of f
                                ["rtol_f"]: relative stopping tolerance for the progress in the values of f
                            ["atol_gradf"]: absolute stopping tolerance for the norm of the gradient of f
                            ["rtol_gradf"]: relative stopping tolerance for the norm of the gradient of f
                        ["max_iterations"]: maximum number of iterations
                             ["verbosity"]: "verbose" or "quiet"
                          ["keep_history"]: whether or not to store the iteration history (True or False) 
                      Here 'norm' refers to the preconditioner-induced norm.
    
  Returns: 
              result: a dictionary containing 
                          solution: final iterate
                          function: the final iterate's objective value 
                          gradient: the final iterate's objective gradient value
                     norm_gradient: preconditioner-induced norm of final objective gradient 
                              iter: number of iterations performed
                          exitflag: flag encoding why the algorithm terminated
                                   0: stopping tolerance described by atol_x, rtol_x, atol_f, rtol_f reached
                                   1: stopping tolerance described by atol_gradf and rtol_gradf reached
                                   2: maximum number of iterations reached

                       history: a dictionary for the history of the run containing
                                          iterates: the iterates x
                                  objective_values: the values of the objective function
                                    gradient_norms: the norms of the objective function gradient 
                                     steps_lengths: the step lengths chosen by the step length rule
  """
  # Define computation of the squared preconditioner norm
  def norm2(d): return d.dot(preconditioner.dot(d))

  # Define an output function that will be used to print information on the state of the iteration
  def print_header(): 
    print('--------------------------------------------------------------------')
    print(' ITER          OBJ    NORM_GRAD    NORM_CORR     OBJ_CHNG           ')
    print('--------------------------------------------------------------------')
  
  # Define exitflags messages that will be printed when the algorithm terminates
  exitflag_messages = [
      'Relative and absolute tolerances on the norm of the update and the descent of the objective are satisfied.',
      'Relative and absolute tolerances on the norm of the gradient are satisfied.',
      'Maximum number of optimization steps is reached.',
      ]
  
  # Get the algorithmic parameters, using defaults if missing
  atol_x = parameters.get("atol_x", 1e-6)
  rtol_x = parameters.get("rtol_x", 1e-6)
  atol_f = parameters.get("atol_f", 1e-6)
  rtol_f = parameters.get("rtol_f", rtol_x**2)
  atol_gradf = parameters.get("atol_gradf", 1e-6)
  rtol_gradf = parameters.get("rtol_gradf", 1e-6)
  max_iterations = parameters.get("max_iterations", 1e3)
  verbosity = parameters.get("verbosity", "quiet")
  keep_history = parameters.get("keep_history", False)

  # Initialize the iterates, counters etc.
  x = x0
  iter = 0
  exitflag = None
  
  # Initialize dummy values pertaining to the previous iterate
  x_old = np.full(x0.shape, np.inf)
  function_value_old = np.inf

  # Prepare a dictionary to store the history
  if keep_history:
    history = {
      "iterates" : [],
      "objective_values" : [],
      "gradient_norms" : [],
      "step_lengths" : []
      }
  
  # Perform gradient descent steps until one of the termination criteria is met
  while exitflag is None:
    # Record the current iterate
    if keep_history: history["iterates"].append(x)
    
    # Dump some output
    if verbosity == 'verbose':
      if (iter%10 == 0): print_header()
      print(' %4d  ' % (iter), end = '')
            
    # Stop when the maximum number of iterations has been reached
    if iter >= max_iterations:
      exitflag = 2
      break
    
    # Compute the function value and derivative at current iterate
    values = f(x, derivatives = [True, True, False])
    function_value = values["function"]
    derivative = values["derivative"]
    
    # Record the current value of the objective
    if keep_history: history["objective_values"].append(function_value)
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (function_value), end = '')
    
    ### TODO BEGIN ###
    # Compute the preconditioned gradient and the square of its (preconditioner-induced) norm 
    # gradient = ...
    # norm2_gradient = ...
    ### TODO END ###
    
    # Check the computed norm square for positivity
    if norm2_gradient < 0:
      raise ValueError('Your preconditioner appears not to be positive definite.')
    else:
      norm_gradient = np.sqrt(norm2_gradient)
    
    # Record the current norm of the gradient
    if keep_history: history["gradient_norms"].append(norm_gradient)

    # Remember the norm of the initial gradient
    if (iter == 0): initial_norm_gradient = norm_gradient
        
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  ' % (norm_gradient), end = '')
    
    ### TODO BEGIN ###
    # Stop when the stopping tolerance on the norm of the gradient has been reached
    # if ...
      exitflag = 1
      break
    ### TODO END ###
    
    # Evaluate the norm of the update step
    norm_delta_x = np.sqrt(norm2(x - x_old))
    
    # Evaluate the change in the objective function values
    delta_f = function_value_old - function_value
    
    # Evaluate the reference values for relative tolerances
    abs_function_value_old = np.abs(function_value_old)
    norm_x_old = np.sqrt(norm2(x_old))
    
    # Dump some output
    if verbosity == 'verbose': print('%11.4e  %11.4e' % (norm_delta_x, -delta_f))
    
    # Stop when the stopping tolerance on the change in the objective and the
    # norm of the update step have been reached
    if (delta_f < atol_f + rtol_f * abs_function_value_old) and\
      (norm_delta_x < atol_x + rtol_x * norm_x_old):
      exitflag = 0
      break
    
    ### TODO BEGIN ###
    # Set the update direction
    # d = ...
    ### TODO END ###
    
    # Prepare the line search function, using the function values of the
    # objective and its derivatives and the chain rule
    def phi(t, derivatives):
      values = f(x + t * d, derivatives)
      if derivatives[1]:
        values["derivative"] = values["derivative"].dot(d)
      if derivatives[2]:
        values["Hessian"] = d.dot(values["Hessian"].dot(d))
      return values
    
    # Prepare some data to pass down to the step length computation rule
    reusables = {
      "phi0" : function_value,
      "dphi0" : -norm2_gradient
      }
    
    # Evaluate the step length t using the step length rule
    t, t_exitflag = step_length_rule(phi, reusables)
    
    # Check whether of not the step length was computed succesfully
    if t_exitflag: raise AssertionError('Step length was not computed succesfully.')
    
    # Record the chosen step length
    if keep_history: history["step_lengths"].append(t)
    
    # Save the current iterate and associated function value for the next iteration
    x_old = x
    function_value_old = function_value
    
    ### TODO BEGIN ###
    # Update the iterate and increase the counter
    # x = ...
    # iter = ...
    ### TODO END ###

  # Dump some output
  if verbosity == 'verbose':
    print('\n\nThe gradient descent method exiting with flag %d.\n' %(exitflag) + str(exitflag_messages[exitflag])+'\n' )
  
  # Create and populate the result to be returned
  result = {
    "solution" : x,
    "function" : function_value,
    "gradient" : gradient,
    "norm_gradient" : norm_gradient,
    "iter" : iter,
    "exitflag" : exitflag
    }

  # Assign the history to the result if required
  if keep_history:
    result["history"] = history
      
  return result


### Implementierung der exakten quadratischen Schrittweite

Um das oben implementierte Gradientenverfahren im Rahmen dieses Abschnitts untersuchen zu können, benötigen wir lediglich noch die Schrittweitensteuerung.

**Aufgabe:** Vervollständigen Sie den Code in der nächsten Zelle und führen Sie die Zelle aus.

Beachten Sie, dass das Interface der Schrittweitenbestimmung lediglich den *Schnitt* $\varphi$ der Funktion $f$ übergeben bekommt. 
Sie müssen also den Ausdruck in Abhängigkeit von dem quadratischen Teil $Q$ der Zielfunktion und dem Vorkonditionierer $M$ aus dem Skript durch durch $\varphi$-Terme ausdrücken.

In [ ]:
# This module implements the exact step length computation for quadratic objective functionals in the gradient scheme

import numpy as np

def exact_step_length_quadratic(phi, reusables = {}):
  """ 
  Compute the exact minimizer of a one-dimensional function assumed
  to be a quadratic polynomial.

  Accepts: 
           phi: evaluates the function the line search is performed on
     reusables: additional information that may be provided to the method (dictionary);
                the following key/value pairs are evaluated:
                  ["phi0"]: the value of phi at t = 0 (scalar)
                  ["dphi0"]: the value of the derivative of phi at t = 0 (scalar)

  Returns:
            t: the step length minimizing phi (provided it is quadratic)
     exitflag: 0
  """

  # The exact minimizer of phi is evaluated using the following data:
  # phi(0), phi'(0), phi(1). Evaluate this data if it is not provided.
  phi0 = reusables.get("phi0", phi(0, derivatives = [True, False, False])["function"]) or\
    phi(0, derivatives = [True,False,False])["function"]
  dphi0 = reusables.get("dphi0", phi(0, derivatives=[False, True, False])["derivative"]) or\
    phi(0, derivatives = [False,True,False])["derivative"]
  if dphi0 >= 0:
    raise(InputError('The function phi is expected to be decreasing at zero..'))
  phi1 = phi(1,derivatives = [True, False, False])["function"]
  
  ### TODO BEGIN ###
  # Evaluate the exact step length
  # t = ...
  ### TODO END ###
  
  
  # Check if the step length is in fact positive, i.e., whether phi has positive curvature.
  if t < 0.0:
    raise ValueError('The step length computation yields a negative step length.')
  else:
    return t, 0

### Untersuchung des Verhaltens der Iterierten

An dieser Stelle können wir bereits mit unserer Untersuchung beginnen.
Das Skript in der nächsten Zelle soll das implementierte Gradientenverfahren mit verschiedenen Vorkonditionierern auf ein quadratisches Problem anwenden und die verallgemeinerten Konditionszahlen berechnen.
Der Output entspricht dem Status des Gradientenverfahren in den jeweiligen Iterationen.
Das Skript ist lauffähig und verwendet die euklidische Vorkonditionierung.

**Aufgabe:** Implementieren Sie zu dem euklidischen Vorkonditionierer mindestens drei weitere Vorkonditionierer.
Achten Sie darauf, auch einen zu implementieren, der zu langsamerer Konvergenz als der euklidische, führt.

In [ ]:
import sys
sys.path.append('src/')

import numpy as np

from objective_functions import *
from scipy.linalg import eigh
from visualization_functions import *

# Create data
Q = np.array([[10.0, -2.0], [-2.0, 1.0]])
c = np.array([2.0, 0.0])
gamma = 0.0
f = lambda u, derivatives: quadratic_function(u, derivatives, Q, c, gamma)
x0 = np.array([1.0, 5.0])

# Construct step length rule
exact_step_length_rule_quadratic = lambda phi, reusables: exact_step_length_quadratic(phi, reusables)

# Construct the preconditioners
preconditioners = [(np.identity(len(x0)), "Identity"), # Identity                  
                   ### TODO BEGIN ###
                   # Add three preconditioners of the format
                   # (matrix, "Label for plotting"),
                   ### TODO END ###
                  ] 

# Set gradient scheme parameters
optimization_parameters = {
"atol_x" : 1e-7,
"rtol_x" : 1e-7,
"atol_f" : 1e-7,
"rtol_f" : 1e-14,
"max_iterations" : 1e4,
"c" : 10,
"verbosity" : "verbose",
"keep_history" : True
}

outputs = []
labels = []
generalized_condition_numbers = []

# Solve problem for all choices of the preconditioner
for preconditioner, label in preconditioners:
  outputs.append(gradient_descent(f, x0, exact_step_length_rule_quadratic, 
                                 preconditioner, optimization_parameters))
  labels.append(label)
  generalized_eigenvalues = eigh(Q, preconditioner, eigvals_only = True)
  generalized_condition_numbers.append(generalized_eigenvalues[-1] / generalized_eigenvalues[0])

Falls der Code in der vorherigen Zelle durchgelaufen ist, können wir nun die Höhenlinien der Zielfunktion und die von den verschiedenen Durchläufen besuchten Iterierten plotten.
Führen Sie dazu die folgende Zelle aus, Sie müssen nichts ergänzen.

In [ ]:
# Plot history in iterate space
plot_2d_iterates_contours(f, list(out["history"] for out in outputs), labels)

**Aufgabe:** Beschreiben Sie den Einfluss der von Ihnen gewählten Vorkonditionierer auf den Verlauf der Iterationen.

**TODO Ihre Antwort hier**

### Untersuchung des Konvergenzverhaltens

Nun wollen wir uns noch das Konvergenzverhalten im Sinne der Energienorm des Fehlers ansehen.
Für unser obiges Problem können wir nämlich den exakten Minimierer direkt bestimmen. 
Mit den Aussagen im Konvergenzbeweis können wir nun die (quadrierte) Energienorm des Fehlers in jeder Iteration gegen eine in dem Vorfaktor $\frac{k-1}{k+1}$ exponentiell fallende Folge abschätzen und als obere Schranke plotten.

**Aufgabe: Berechnen Sie den exakten Minimierer mittels der Bedingungen erster Ordnung.**

In [ ]:
### TODO BEGIN ###
# Compute the actual solution of the problem and its function value for verification
# x_opt = ...
f_opt = f(x_opt, derivatives = [True, False, False])["function"]
### TODO END ###

# Plot functional value differences (approximation of error energy norm)
plot_f_val_diffs(list(out["history"] for out in outputs), 
                [f_opt] * len(outputs),
                labels,
                generalized_condition_numbers)

**Aufgabe:** Beschreiben Sie kurz das beobachtete Verhalten in diesem Plot und ob er mit den Ergebnissen aus der Vorlesung konsistent ist.

**TODO Ihre Antwort hier**

### Untersuchung der Schrittweiten und der Gradientennormen

Als Referenz für Sie, plotten wir mit der nächsten Zelle noch einmal die gewählten Schrittweiten und die Vorkonditionierernorm der Gradienten.
Führen Sie dafür die Zelle einfach aus.
Beachten Sie das stark zappelige Verhalten.

In [ ]:
plot_step_sizes(list(out["history"] for out in outputs), labels)

plot_grad_norms(list(out["history"] for out in outputs), labels)

## Das Gradientenverfahren für nichtquadratische Funktionen

Quadratische Funktionen sind in der unbeschränkten Optimierung die einfachsten, sinnvoll zu untersuchenden Funktionen. 
Für Optimierungsaufgaben mit allgemeineren Funktionen ist das Verhalten des Gradientenverfahrens nicht mehr so leicht zu analysieren und vorherzusagen.
Außerdem ist die Wahl eines "nützlichen" Vorkonditionierers entsprechend schwieriger.
Wir werden jetzt das Verhalten des Gradientenverfahrens mit verschiedenen Vorkonditionierern und Armijo-Backtracking anhand der [Funktion von *Himmelblau*](https://en.wikipedia.org/wiki/Himmelblau%27s_function) - insbesondere hinsichtlich des Verhaltens bzgl. lokaler Minimierer untersuchen.


### Armijo Backtracking Strategie

Das Gradientenverfahren von oben können wir natürlich weiterverwenden.
Lediglich die Schrittweitensteuerung müssen wir austauschen, denn für allgemeine Funktionen finden wir keinen analytischen Ausdruck für eine exakte Schrittweite.
Wir werden stattdessen die im Skript beschriebene Backtracking Strategie mit der Armijo Regel implementieren.

**Aufgaben:** Vervollständigen Sie den Code zur Armijo Schrittweitensteuerung in der nächsten Zelle.

In [ ]:
def armijo_backtracking(phi, reusables = {}, parameters = {}):
  """
  Compute a step length t via backtracking satisfying the Armijo condition 
  for the function phi, i.e.,

    phi(t) <= phi(0) + sigma * t * dphi(0) 

  where dphi(0) is the derivative of phi at t = 0.

  Accepts: 
           phi: evaluates the function the line search is performed on
     reusables: additional information that may be provided to the method (dictionary);
                the following key/value pairs are evaluated:
                   ["phi0"]: the value of phi at t = 0 (scalar)
                  ["dphi0"]: the value of the derivative of phi at t = 0 (scalar)

  Returns:
            t: the step length minimizing phi (provided it is quadratic)
     exitflag: flag encoding why the line search terminated
                 0: success
                 1: maximum number of iterations reached
                 2: trial step length became too small
  """

  def print_header():
    print('--------------------------------------------------------------------')
    print(' ARMIJO:     ITER          STEP      OBJCHNG           ')
    print('--------------------------------------------------------------------')
  
  # Get the line search parameters, using defaults if missing
  sigma = parameters.get("sigma", 0.01)
  beta = parameters.get("beta", 0.5)
  initial_t = parameters.get("initial_step_length", 1.0)
  verbosity = parameters.get("verbosity", "quiet")
  max_iterations = parameters.get("max_iterations", 1e4)
    
  # Extract or compute required data for checking armijo condition
  phi0 = reusables.get("phi0", phi(0, derivatives = [True, False, False])["function"]) or\
    phi(0, derivatives[True,False,False])["function"]
  dphi0 = reusables.get("dphi0", phi(0, derivatives = [False, True, False])["derivative"]) or\
    phi(0, derivatives[False,True,False])["derivative"]
  
  if dphi0 >= 0:
    raise(InputError('The function phi is expected to be decreasing at zero..'))
  
  # Initialize the step length and counter
  t = initial_t
  iter = 0
  exitflag = None
    
  # Perform the backtracking search until one of the termination criteria is met
  while exitflag is None:

    # Evaluate the value of phi at the current trial step length and the amount of descent
    phi_trial = phi(t, derivatives = [True, False, False])["function"]
    delta_phi = phi_trial - phi0
    
    # Dump some output
    if verbosity == 'verbose':
      if (iter%10 == 0): print_header()
      print('             %4d   %11.4e  %11.4e  \n' % (iter, t, delta_phi))
    
    # Verify the Armijo condition
    ### TODO BEGIN ###
    # if ...
      exitflag = 0
      break
    ### TODO END ###
    
    # Stop when the maximum number of iterations has been reached
    elif iter >= max_iterations:
      exitflag = 1
      print('Warning: Armijo is stopping because the maximum number of iterations is reached.\n')
      break
    # Stop when the function appears locally constant and the initial step length has decreased significantly
    elif (delta_phi == 0) and (t / initial_t < 1e-12): 
      exitflag = 2                               
      if verbosity == 'verbose':
        print('Warning: Armijo is stopping because the function appears locally constant.\n')
      break
    
    ### TODO BEGIN ###
    # Reduce the trial step size and increase the counter
    # t = ...
    # iter = ...
    ### TODO END ###
    
  # Check whether the step length is in fact positive
  if t < 0.0:
    raise ValueError('Armijio is returning a negative step length.')
  else:
    return t, exitflag

### Lokale Minimierer

Das Gradientenverfahren arbeitet mit lokalen Informationen.
Für nichtquadratische Funktionen, z.B. welche die mehrere stationäre Punkte haben, können wir nicht vorhersagen ob bzw. zu welchem Punkt das Verfahren konvergieren wird.
Im folgenden Skript soll das Gradientenverfahren für verschiedene Vorkonditionierer auf die Minimierung der Himmelblaufunktion angewendet und die Ergebnisse geplottet werden.

**Aufgabe:** Implementieren Sie zwei weitere Vorkonditionierer, nämlich:
1. Den Vorkonditionierer, der sich durch die Wahl der Hessematrix an der Startiterierten ergibt. (Der erste Schritt ist also ein Newton-Schritt)
1. Den Vorkonditionierer, der sich durch die Wahl der Hessematrix an einem Minimierer (3,2) ergibt.

In [ ]:
import sys
sys.path.append('src/')

import numpy as np

from objective_functions import *
from visualization_functions import *

# Create problem data
f = himmelblau
x0 = np.array([8.0, 0.0])

# Set parameters for armijo linesearch
armijo_parameters = {
"sigma" : 0.01,
"beta" : 0.5,
"initial_step_length" : 2,
#"verbosity" : "verbose"
}

# Construct step length rule
armijo_step_length_rule = lambda phi, reusables: armijo_backtracking(phi, reusables, armijo_parameters);

# Construct the preconditioners
preconditioners = [(np.identity(len(x0)), "Identity"),
                   ### TODO BEGIN ###
                   # Add two preconditioners of the format
                   # (matrix, "Label for plotting"),
                   ### TODO END ###
                  ]

# Set gradient scheme parameters
optimization_parameters = {
"atol_x" : 1e-7,
"rtol_x" : 1e-7,
"atol_f" : 1e-7,
"rtol_f" : 1e-14,
"max_iterations" : 1e4,
"c" : 10,
"verbosity" : "verbose",
"keep_history" : True
}

outputs = []
labels = []

# Solve problem for all preconditioners
for preconditioner, label in preconditioners:
  outputs.append(gradient_descent(f, x0, armijo_step_length_rule, preconditioner, optimization_parameters))
  labels.append(label)
    
# Plot history in iterate space
plot_2d_iterates_contours(f, list(out["history"] for out in outputs), labels)

# Plot functional value differences (approximation of error energy norm)
plot_f_val_diffs(list(out["history"] for out in outputs), 
                list(out["history"]["objective_values"][-1] for out in outputs),
                labels)

plot_step_sizes(list(out["history"] for out in outputs), labels)

plot_grad_norms(list(out["history"] for out in outputs), labels)

**Aufgabe:** Beschreiben Sie Ihre Beobachtungen zum Verhalten des obigen Beispiels. Variieren Sie auch den Startpunkt $x_0$ (Zeile 12) und erklären Sie, warum die Wahl der Hessematrix an der Startiterierten keine sonderlich gute Idee ist.

**TODO Ihre Antwort hier.**